# Implementation of a new rPPG method

## Part 2 : Notebook for the prediction of the 3D-CNN model

This jupyter notebook file complements the "Train_3DCNN_model_BPM.ipynb" file. In this file, we can test the model predictions on real videos and highlight logic of the future implementation into the pyVHR framework. ([Link](https://ieeexplore.ieee.org/document/9272290)) ([GitHub](https://github.com/phuselab/pyVHR))

This file is based on the implementation described in the following article :
Frédéric Bousefsaf, Alain Pruski, Choubeila Maaoui, 3D convolutional neural networks for remote pulse rate measurement and mapping from facial video, Applied Sciences, vol. 9, n° 20, 4364 (2019). ([Link](https://www.mdpi.com/2076-3417/9/20/4364)) ([GitHub](https://github.com/frederic-bousefsaf/ippg-3dcnn))

## Importing libraries

Previously , you have to install theses python librairies :
* tensorflow
* matplotlib
* scipy
* numpy
* opencv-python
* Copy
* pyVHR

In [1]:
import os
#RUN ON CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#Tensorflow/KERAS
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.utils import np_utils

# Numpy / Matplotlib / OpenCV / Scipy / Copy
import numpy as np
import scipy.io
import scipy.stats as sp
import matplotlib.pyplot as plt
import cv2
from copy import copy

#pyVHR
from pyVHR.signals.video import Video
from pyVHR.datasets.dataset import Dataset
from pyVHR.datasets.dataset import datasetFactory

# Functions for making predictions

## Loading the video & pyVHR processing


In the pyVHR framework, we work on a processed video. The processing consists of detecting and extracting an area of interest, in order to apply our rPPGs methods on relevant data.

In [2]:
# -- Video object
def extractionROI(videoFilename):
    video = Video(videoFilename)
    video.getCroppedFaces(detector='dlib', extractor='skvideo')
    video.setMask(typeROI='skin_adapt',skinThresh_adapt=0.20)
    return video

## Loading the model
Load model & classes

In [3]:
# Load model
def loadmodel(MODEL_PATH):
    # load data in files
    model = model_from_json(open(f'{MODEL_PATH}/model_conv3D.json').read())
    model.load_weights(f'{MODEL_PATH}/weights_conv3D.h5')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # define the frequencies // output dimension (number of classes used during training)
    freq_BPM = np.linspace(55, 240, num=model.output_shape[1]-1)
    freq_BPM = np.append(freq_BPM, -1)     # noise class
    return model, freq_BPM


## Converting videoframes to a single channel array

Select one channel for making prediction

In [4]:
# 2. LOAD DATA
def convertVideoToTable(video,model, startFrame):
    imgs = np.zeros(shape=(model.input_shape[1], video.cropSize[0], video.cropSize[1], 1))

    # channel extraction
    if (video.cropSize[2]<3):
        IMAGE_CHANNELS = 1
    else:
        IMAGE_CHANNELS = video.cropSize[2]

    # load images (imgs contains the whole video)
    for j in range(0, model.input_shape[1]):

        if (IMAGE_CHANNELS==3):
            temp = video.faces[j + startFrame]/255
            temp = temp[:,:,1]      # only the G component is currently used
        else:
            temp = video.faces[j + startFrame] / 255

        imgs[j] = np.expand_dims(temp, 2)
    return imgs

## Formating Video

Realization of a catagraphy of predictions on the video.
This function formats the video in several sets of tests, in order to make multiple predictions. The sum of these predictions is returned.

In [5]:
def formatingDataTest(video, model, imgs, freq_BPM, stepX, stepY):
    
    # output - sum of predictions
    predictions = np.zeros(shape=(len(freq_BPM)))
    
    # Displacement on the x axis
    iterationX = 0
    # Our position at n + 1 on the X axis
    axisX = model.input_shape[3]
    
    # width of video
    width = video.cropSize[1]
    # height of video
    height = video.cropSize[0]
    
    # Browse the X axis
    while axisX < width:
        # Displacement on the y axis
        axisY = model.input_shape[2]
        # Our position at n + 1 on the Y axis
        iterationY = 0
        # Browse the Y axis
        while axisY < height:
            
            # Start position
            x1 = iterationX * stepX
            y1 = iterationY * stepY
            
            # End position
            x2 = x1 + model.input_shape[3]
            y2 = y1 + model.input_shape[2]
            
            # Cutting 
            faceCopy = copy(imgs[0:model.input_shape[1],x1:x2,y1:y2,:])
            
            # randomize pixel locations
            for j in range(model.input_shape[1]):
                temp = copy(faceCopy[j,:,:,:])
                np.random.shuffle(temp)
                faceCopy[j] = temp
            
            # Checks the validity of cutting
            if(np.shape(faceCopy)[1] == model.input_shape[3] and np.shape(faceCopy)[2] == model.input_shape[2]):
                # prediction on the cut part
                xtest = faceCopy - np.mean(faceCopy)
                predictions = predictions + getPrediction(model,freq_BPM,xtest)
            
            # increments
            axisY = y2 + model.input_shape[2]
            iterationY = iterationY +1
        # increments    
        axisX = x2 + model.input_shape[3]
        iterationX = iterationX + 1
        
    return predictions        
    


## Making a prediction

Use the model to make prediction 

In [6]:
def getPrediction(model,freq_BPM, xtest):
    idx =0
    maxi =0
    # model.predict
    input_tensor = tf.convert_to_tensor(np.expand_dims(xtest, 0))
    h = model(input_tensor)
    h = h.numpy() 
    # convert prediction to binary
    res = np.zeros(shape=(76))
    idx = getIdx(h[0])
    res[idx] = 1
    return res

## Get the index of the maximum value of a prediction

In [7]:
def getIdx(h):
    idx =0
    maxi =-1
    for i in range(0, len(h)):
        if maxi < h[i]:
            idx = i
            maxi = h[i]
    return idx  

## Finding the label associated with the prediction

In [30]:
def getClass(h, freq_BPM, interval): 
    # index in the list
    iterator = 0
    # maximum value of an interval
    sum_max = -1
    # index of maximum value of an interval
    idx_max = 0
    # security against list out of range problems
    out_of_range = len(freq_BPM) -1
    
    # Calculating the sum of all intervals
    while iterator < (len(h)- interval - 1):
        # value of studied interval
        sum_interval = 0
        # representative index of value of studied interval
        idx_interval = iterator+ int(interval/2)
        
        # Calculates variables
        for j in range(interval):
            sum_interval = sum_interval + h[iterator + j]
        
        # Case where value of the studied interval equal to the value of the maximum interval
        if sum_interval == sum_max and sum_interval != 0:
            # Decision based on representative indexes
            if(h[idx_interval] > h[idx_max]):
                sum_max = sum_interval
                idx_max = idx_interval
            # Decision based on neighbors of representative indexes    
            elif(h[idx_interval] == h[idx_max]):
                sum_max, idx_max = study_neighborhoods(h, sum_max,sum_interval, idx_max, idx_interval, out_of_range)

        # Case where the studied interval is better than the maximum interval                
        elif (sum_interval > sum_max):
            sum_max = sum_interval
            idx_max = idx_interval
        #increment    
        iterator = iterator +1
        
    # return bpm value   
    return freq_BPM[idx_max] 

def study_neighborhoods(h, sum_max,sum_interval, idx_max, idx_interval, out_of_range):
    # Decision statue
    find = False
    # Neighborhood degree
    var = 1
    # intervals according to the degree of neighborhood
    sum_var_max = h[idx_max]
    sum_var_interval = h[idx_interval]
                
    # Study of neighborhood intervals
    while find is not True:
                    
        if(idx_interval + var < out_of_range):
            sum_var_interval = sum_var_interval + h[idx_interval + var]
            
        if(idx_interval - var >= 0):
            sum_var_interval = sum_var_interval + h[idx_interval - var]
            
        if(idx_max + var < out_of_range):
            sum_var_max = sum_var_max + h[idx_max + var]
            
        if(idx_max - var >= 0):
            sum_var_max = sum_var_max + h[idx_max - var]
                       
        if(sum_var_max < sum_var_interval):
            sum_max = sum_interval
            idx_max = idx_interval
            find =True
            
        if(sum_var_max > sum_var_interval):
            find =True
            
        if(var > 10):
            find = True  
        else :
            var = var + 1
        
    return sum_max, idx_max


# Make a prediction

Function to make prediction on veritable data (150 first frames only in this example)

In [34]:
videoFilename = "./UBFC/DATASET_2/subject1/vid.avi"  #video to be processed path
modelFilename = "./final_script/model/"   #model path 

def makePrediction(videoFilename, modelFilename):
    # ROI EXTRACTION
    video = extractionROI(videoFilename)
    # print ROI EXTRACTION
    video.showVideo()  
    #Load the model
    model, freq_BPM = loadmodel(modelFilename)
    #extract Green channel or Black & whrite channel
    framesOneChannel = convertVideoToTable(video,model,0)
    #Data preparation 
    Xstep = 5
    Ystep = 5
    prediction = formatingDataTest(video, model, framesOneChannel, freq_BPM, Xstep, Ystep)
    print(prediction)
    bpm = getClass(prediction, freq_BPM, 6)
    return bpm


print('BPM frequency estimated = ' + str(makePrediction(videoFilename, modelFilename)))

interactive(children=(IntSlider(value=1, description='frame', max=1533, min=1), Output()), _dom_classes=('widg…

[  0.   0.   0.   1.   0.   1.   3.   3.   2.   4.  13.   1.   3.   1.
   0.   4.  11.   0.   0.   0.   0.   0.   2.   0.   0.   0.   0.   0.
   2.   1.   3.   0.   1.   0.   0.   2.   0.   0.   0.   0.  21.   0.
   0.   0.   0.   0.   0.   4.   0.   0.   0.   0.   0.   0.   3.   2.
   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   4.   1.  17.
   1.   0.   0.   0.   0. 173.]
BPM frequency estimated = 87.5


# Validation test on veritable data

Test on 150 first frames

In [35]:
# video filenames
videoFilenames = ["./UBFC/DATASET_2/subject1/vid.avi", "./UBFC/DATASET_2/subject33/vid.avi"]
# Ground Truth (GT) filenanmes
GT = ["./UBFC/DATASET_2/subject1/ground_truth.txt","./UBFC/DATASET_2/subject33/ground_truth.txt"]
# model filename
modelFilename = "./model/"
# load model
model, freq_BPM = loadmodel(modelFilename)
# load dataset of videos
dataset = datasetFactory("UBFC2")
# Window size GT
winSizeGT = 5      

# For each videos
for i in range(0, len(videoFilenames)):
    # prediction by model
    prediction = makePrediction(videoFilenames[i], modelFilename)
    print("Prediction Video "+ str(i+1) +" : "+ str(prediction))
    # Reality
    sigGT = dataset.readSigfile(GT[i])
    bpmGT, timesGT = sigGT.getBPM(winSizeGT)
    # Format the GT
    bpm = np.round(bpmGT)
    bpm = bpm - 55
    bpm = np.round(bpm / 2.5)
    GT_value = freq_BPM[int(bpm[2])]
    print("GT Video "+ str(i+1) +" : "+str(GT_value))
    # difference
    print("ABS DIFF Video "+ str(i+1) +" : "+str(abs(GT_value-prediction)))
    

interactive(children=(IntSlider(value=1, description='frame', max=1533, min=1), Output()), _dom_classes=('widg…

[  0.   0.   1.   5.   3.   1.   0.   1.   1.   5.  21.   3.   5.   2.
   3.   5.  12.   0.   2.   0.   0.   0.   0.   2.   0.   0.   0.   0.
   0.   2.   1.   0.   0.   0.   0.   3.   0.   0.   0.   1.  31.   0.
   0.   0.   1.   0.   0.   3.   0.   0.   0.   0.   0.   0.   1.   1.
   0.   1.   0.   1.   0.   0.   1.   0.   0.   0.   2.   3.   1.  13.
   2.   0.   0.   0.   0. 145.]
Prediction Video 1 : 87.5
GT Video 1 : 92.5
ABS DIFF Video 1 : 5.0


interactive(children=(IntSlider(value=1, description='frame', max=1984, min=1), Output()), _dom_classes=('widg…

[  0.   0.   1.   0.   1.   4.   0.   0.   0.   0.   0.   0.   0.   1.
   2.   0.   1.   0.   0.   0.   0.   0.   2.   9.   1.   1.   1.   2.
   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   1.
   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.
   0.   0.   1.   0.   0. 132.]
Prediction Video 2 : 117.5
GT Video 2 : 115.0
ABS DIFF Video 2 : 2.5
